In [27]:
import gym
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
import gym
import matplotlib.pyplot as plt
import matplotlib.animation
from IPython.display import HTML

# Neural Network for Q-learning
class QNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

# Parameters
learning_rate = 0.001
gamma = 0.99  # discount factor
epsilon = 0.1  # exploration rate
num_episodes = 1000

# Environment setup
env = gym.make('CartPole-v1')
input_size = env.observation_space.shape[0]
output_size = env.action_space.n

# Q-Network
net = QNetwork(input_size, 64, output_size)
optimizer = optim.Adam(net.parameters(), lr=learning_rate)
loss_fn = nn.MSELoss()
frames = []


# Training loop
# Training loop
for episode in range(num_episodes):
    state, _ = env.reset()  # Unpack the state to ignore the dictionary
    done = False
    loss = None  # Initialize loss

    while not done:
        frames.append(env.render())
        state_tensor = torch.FloatTensor([state]).view(1, -1)
        with torch.no_grad():
            q_values = net(state_tensor)

        # Epsilon-greedy strategy
        if random.random() < epsilon:
            action = env.action_space.sample()  # Explore
        else:
            action = torch.argmax(q_values).item()  # Exploit

        # Take action
        step_output = env.step(action)
        try:
            next_state, reward, done, _, _ = step_output  # Adjust unpacking for extra element
        except ValueError as e:
            print(f"Error unpacking step output: {e}")
            break

        next_state_tensor = torch.FloatTensor([next_state]).view(1, -1)
        with torch.no_grad():
            next_q_values = net(next_state_tensor)

        # Ensure q_target has the same shape as q_val
        q_target = reward + gamma * torch.max(next_q_values).detach()  # Detach target from graph
        q_val = q_values.gather(1, torch.tensor([[action]], device=q_values.device))  # Ensure q_val is part of the graph and has the correct shape

        q_target.requires_grad_(True)
        q_val.requires_grad_(True)
        
        # Backpropagation
        optimizer.zero_grad()
        loss = loss_fn(q_val, q_target.unsqueeze(-1))  # Make sure both tensors have the same shape
        loss.backward()
        optimizer.step()



        state = next_state

    if episode % 100 == 0 and loss is not None:
        print(f'Episode: {episode}, Loss: {loss.item()}')

env.close()






c:\Users\pari0\anaconda3\envs\simulator\lib\site-packages\gym\envs\classic_control\cartpole.py:211: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym("CartPole-v1", render_mode="rgb_array")
  gym.logger.warn(
c:\Users\pari0\anaconda3\envs\simulator\lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
c:\Users\pari0\anaconda3\envs\simulator\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Episode: 0, Loss: 1.02150559425354
Episode: 100, Loss: 1.0225005149841309
Episode: 200, Loss: 1.023775577545166
Episode: 300, Loss: 1.3533390760421753
Episode: 400, Loss: 1.0166887044906616
Episode: 500, Loss: 1.0237770080566406
Episode: 600, Loss: 1.0226658582687378
Episode: 700, Loss: 1.022606611251831
Episode: 800, Loss: 1.0236257314682007
Episode: 900, Loss: 1.0228310823440552


In [36]:
import gymnasium as gym
import math
import random
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

env = gym.make("CartPole-v1")

# matplotlib 설정
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

# GPU를 사용할 경우
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu
